<a href="https://colab.research.google.com/github/jeremiahoclark/python-coding-patterns/blob/main/04_concurrency_parallelism_patterns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 4. Concurrency and Parallelism Patterns

Writing concurrent (multithreaded or async) and parallel (multi-process or multi-core) code in Python can greatly enhance performance for I/O-bound tasks and utilize multiple CPUs for CPU-bound tasks. Python offers several concurrency models: threads, processes, async IO, and more. Here we'll cover patterns such as Producer-Consumer, Thread Pool, futures, and async/await usage.

**Note:** Due to the Global Interpreter Lock (GIL) in CPython, threads do not run Python bytecode in true parallel on multiple cores; they are best for I/O-bound tasks. For CPU-bound tasks, multiple processes or releasing the GIL via C extensions (e.g., numpy) is used. We'll see patterns for both scenarios.

## 4.1 Producer-Consumer Pattern (Thread-based)

**Pattern Profile:**

- **Name:** Producer-Consumer
- **Category:** Concurrency Design Pattern
- **Difficulty:** Intermediate
- **Python Version:** 3.x (uses `queue.Queue`, `threading`)
- **Dependencies:** `threading`, `queue` (std lib)

**Problem Statement:** When one part of a program produces data and another part consumes it, coordinating them can be challenging. The producer-consumer pattern decouples production and consumption using a thread-safe queue. Producers put items into the queue; consumers take items out. This smooths out differences in production/consumption rates and avoids the producer overwriting data that hasn't been processed.

**Solution Approach:** Use a `queue.Queue` (thread-safe, blocking) for communication. Start producer thread(s) that read or generate data and `.put()` into the queue. Start consumer thread(s) that `.get()` from the queue and process data. Use queue blocking operations or a sentinel value to signal completion.

In [ ]:
import threading
import queue
import time
import random

# Create a bounded queue
q = queue.Queue(maxsize=5)  # bounded buffer of size 5

def producer(name, count):
    """Producer function that generates items and puts them in the queue"""
    for i in range(count):
        item = f"{name}-item{i}"
        q.put(item)  # will block if queue is full
        print(f"[Producer {name}] produced {item}")
        time.sleep(random.random()*0.1)  # simulate work
    q.put(None)  # sentinel for end of production
    print(f"[Producer {name}] done.")

def consumer(name):
    """Consumer function that processes items from the queue"""
    while True:
        item = q.get()  # will block if queue is empty
        if item is None:
            # put back sentinel for other consumers and exit
            q.put(None)
            print(f"(Consumer {name} sees shutdown signal)")
            break
        print(f"--> [Consumer {name}] consuming {item}")
        time.sleep(random.random()*0.2)  # simulate processing
        q.task_done()

# Example usage - comment out when running other examples
if __name__ == "__main__":
    # Launch one producer and two consumers
    producer_thread = threading.Thread(target=producer, args=("P1", 10), daemon=True)
    consumer_threads = [threading.Thread(target=consumer, args=(f"C{i}",), daemon=True) for i in (1,2)]

    producer_thread.start()
    for ct in consumer_threads:
        ct.start()

    producer_thread.join()
    q.join()  # wait until all items including sentinel are processed
    print("All done.")

**Advanced Producer-Consumer Example with Multiple Producers:**

In [ ]:
import threading
import queue
import time
import random

class ProducerConsumerDemo:
    def __init__(self, queue_size=10):
        self.queue = queue.Queue(maxsize=queue_size)
        self.active_producers = 0
        self.lock = threading.Lock()

    def producer(self, name, count):
        """Enhanced producer with proper shutdown coordination"""
        with self.lock:
            self.active_producers += 1

        try:
            for i in range(count):
                item = f"{name}-item{i}"
                self.queue.put(item)
                print(f"[Producer {name}] produced {item}")
                time.sleep(random.uniform(0.01, 0.1))  # simulate work
        finally:
            with self.lock:
                self.active_producers -= 1
                if self.active_producers == 0:
                    # Last producer signals end
                    self.queue.put(None)
                    print(f"[Producer {name}] is last producer, signaling end")
                else:
                    print(f"[Producer {name}] done, {self.active_producers} producers still active")

    def consumer(self, name):
        """Enhanced consumer with graceful shutdown"""
        processed = 0
        while True:
            try:
                item = self.queue.get(timeout=1)  # timeout to avoid infinite wait
                if item is None:
                    print(f"(Consumer {name}) received shutdown signal after processing {processed} items")
                    self.queue.put(None)  # pass signal to other consumers
                    break

                print(f"--> [Consumer {name}] consuming {item}")
                time.sleep(random.uniform(0.05, 0.15))  # simulate processing
                processed += 1
                self.queue.task_done()

            except queue.Empty:
                print(f"(Consumer {name}) timed out waiting for items")
                continue

    def run_demo(self, num_producers=3, num_consumers=2, items_per_producer=5):
        """Run the producer-consumer demo"""
        print(f"Starting demo with {num_producers} producers, {num_consumers} consumers")

        # Create and start producer threads
        producer_threads = []
        for i in range(num_producers):
            thread = threading.Thread(target=self.producer, args=(f"P{i+1}", items_per_producer))
            producer_threads.append(thread)
            thread.start()

        # Create and start consumer threads
        consumer_threads = []
        for i in range(num_consumers):
            thread = threading.Thread(target=self.consumer, args=(f"C{i+1}",))
            consumer_threads.append(thread)
            thread.start()

        # Wait for all producers to finish
        for thread in producer_threads:
            thread.join()

        # Wait for all consumers to finish
        for thread in consumer_threads:
            thread.join()

        print("Demo completed!")

# Run the demo
demo = ProducerConsumerDemo()
demo.run_demo(num_producers=2, num_consumers=3, items_per_producer=4)

## 4.2 Thread Pool and Futures

**Pattern Profile:**

- **Name:** ThreadPoolExecutor (Thread Pool Pattern)
- **Category:** Concurrency Pattern
- **Difficulty:** Beginner
- **Python Version:** 3.2+ (`concurrent.futures`)
- **Dependencies:** `concurrent.futures.ThreadPoolExecutor`

**Problem Statement:** Creating and destroying threads for many short tasks is expensive. A thread pool reuses a fixed number of threads to execute many tasks. In Python, the `concurrent.futures.ThreadPoolExecutor` provides a high-level interface to submit callables (tasks) to a pool of threads and get back a **Future** object representing the pending result. This pattern is useful for parallelizing I/O-bound operations easily.

**Solution Approach:** Use `ThreadPoolExecutor` with a certain `max_workers`. Submit tasks (callable + arguments) using `executor.submit(fn, *args)`, which returns a Future. A Future can be used to check status or retrieve the result (with `.result()` blocking until done or `.add_done_callback`). Alternatively, use `executor.map(fn, iterable)` to map a function over an iterable in multiple threads, returning results as they come.

In [ ]:
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor, as_completed
import time
import random
import requests  # Note: you may need to install requests

# Simulated I/O-bound task
def simulate_io_task(task_id, duration):
    """Simulate an I/O-bound task like reading a file or making a network request"""
    print(f"Task {task_id} starting (will take {duration:.2f}s)")
    time.sleep(duration)  # Simulate I/O wait
    result = f"Task {task_id} completed after {duration:.2f}s"
    print(f"Task {task_id} finished")
    return result

# Example 1: Using submit() and as_completed()
def thread_pool_submit_example():
    print("=== ThreadPool with submit() example ===")
    tasks = [(i, random.uniform(0.5, 2.0)) for i in range(5)]

    start_time = time.time()

    with ThreadPoolExecutor(max_workers=3) as executor:
        # Submit all tasks
        futures = {executor.submit(simulate_io_task, task_id, duration): task_id
                  for task_id, duration in tasks}

        # Process results as they complete
        for future in as_completed(futures):
            task_id = futures[future]
            try:
                result = future.result()
                print(f"Got result: {result}")
            except Exception as e:
                print(f"Task {task_id} generated an exception: {e}")

    end_time = time.time()
    print(f"Total time: {end_time - start_time:.2f}s\n")

thread_pool_submit_example()

In [ ]:
# Example 2: Using map() for simpler cases
def thread_pool_map_example():
    print("=== ThreadPool with map() example ===")

    def process_item(item):
        """Process a single item"""
        time.sleep(random.uniform(0.1, 0.5))
        return f"Processed {item} -> {item.upper()}"

    items = ["apple", "banana", "cherry", "date", "elderberry"]

    start_time = time.time()

    with ThreadPoolExecutor(max_workers=3) as executor:
        # map() returns results in the same order as input
        results = list(executor.map(process_item, items))

    end_time = time.time()

    for result in results:
        print(result)

    print(f"Total time: {end_time - start_time:.2f}s\n")

thread_pool_map_example()

In [ ]:
# Example 3: CPU-bound task with ProcessPoolExecutor
def cpu_intensive_task(n):
    """A CPU-intensive task - calculating prime numbers"""
    def is_prime(num):
        if num < 2:
            return False
        for i in range(2, int(num ** 0.5) + 1):
            if num % i == 0:
                return False
        return True

    # Find prime numbers up to n
    primes = [i for i in range(2, n) if is_prime(i)]
    return len(primes)

def compare_thread_vs_process():
    print("=== Comparing ThreadPool vs ProcessPool for CPU-bound tasks ===")

    numbers = [10000, 15000, 20000, 25000]

    # Using ThreadPoolExecutor (limited by GIL for CPU-bound)
    print("Using ThreadPoolExecutor:")
    start_time = time.time()
    with ThreadPoolExecutor(max_workers=4) as executor:
        thread_results = list(executor.map(cpu_intensive_task, numbers))
    thread_time = time.time() - start_time
    print(f"Results: {thread_results}")
    print(f"Thread time: {thread_time:.2f}s")

    # Using ProcessPoolExecutor (can utilize multiple cores)
    print("\nUsing ProcessPoolExecutor:")
    start_time = time.time()
    with ProcessPoolExecutor(max_workers=4) as executor:
        process_results = list(executor.map(cpu_intensive_task, numbers))
    process_time = time.time() - start_time
    print(f"Results: {process_results}")
    print(f"Process time: {process_time:.2f}s")

    print(f"\nSpeedup with processes: {thread_time / process_time:.2f}x")

compare_thread_vs_process()

**Advanced Futures Example with Callbacks:**

In [ ]:
from concurrent.futures import ThreadPoolExecutor
import time
import random

def task_with_callback_demo():
    print("=== Futures with Callbacks Example ===")

    def long_running_task(task_id):
        """A task that might succeed or fail"""
        duration = random.uniform(1, 3)
        time.sleep(duration)

        # Randomly fail some tasks
        if random.random() < 0.3:
            raise Exception(f"Task {task_id} failed randomly")

        return f"Task {task_id} succeeded after {duration:.2f}s"

    def success_callback(future):
        """Called when a task completes successfully"""
        try:
            result = future.result()
            print(f"✅ SUCCESS: {result}")
        except Exception as e:
            print(f"❌ FAILED: {e}")

    # Submit tasks with callbacks
    with ThreadPoolExecutor(max_workers=3) as executor:
        futures = []

        for i in range(5):
            future = executor.submit(long_running_task, i)
            future.add_done_callback(success_callback)
            futures.append(future)

        # Wait for all to complete
        for future in futures:
            try:
                future.result()  # This will re-raise any exceptions
            except Exception:
                pass  # Already handled by callback

    print("All tasks completed!")

task_with_callback_demo()

## 4.3 Async/Await and Event Loop Pattern

**Pattern Profile:**

- **Name:** Asyncio Event Loop (async/await)
- **Category:** Concurrency (asynchronous I/O pattern)
- **Difficulty:** Advanced
- **Python Version:** 3.5+ (async/await syntax)
- **Dependencies:** `asyncio`

**Problem Statement:** Traditional threading can have overhead and isn't ideal for high-number-of-connection I/O tasks (e.g., thousands of sockets). Async IO uses an event loop (single-threaded) that cooperatively multitasks across awaiting tasks, suitable for many concurrent I/O operations without the overhead of threads. The pattern is essentially an implementation of the **Reactor pattern** (event loop that waits for events and dispatches handlers) combined with language support via `async/await` to make it easier to write sequential-looking code.

**Solution Approach:** Use `asyncio` library:

- Define `async def` coroutines that use `await` to yield control when performing I/O
- Get an event loop (automatically with `asyncio.run(main())` in Python 3.7+ or manually)
- Schedule tasks with `asyncio.create_task(coro)` if needed to run concurrently within an async function
- The event loop will run, awaiting I/O completion and switching between tasks

In [ ]:
import asyncio
import random
import time

# Basic async producer-consumer example
async def async_producer(queue: asyncio.Queue, count: int):
    """Async producer that generates items"""
    for i in range(count):
        await asyncio.sleep(random.random()*0.1)  # simulate async work
        item = f"item{i}"
        await queue.put(item)
        print(f"[Async Producer] produced {item}")
    await queue.put(None)  # sentinel
    print("[Async Producer] done producing.")

async def async_consumer(queue: asyncio.Queue, name: str):
    """Async consumer that processes items"""
    while True:
        item = await queue.get()
        if item is None:
            await queue.put(None)  # pass sentinel along for other consumers
            print(f"(Async Consumer {name}) sees shutdown signal")
            break
        print(f"--> (Async Consumer {name}) consuming {item}")
        await asyncio.sleep(random.random()*0.2)  # simulate async processing

async def async_producer_consumer_demo():
    """Demo of async producer-consumer pattern"""
    print("=== Async Producer-Consumer Demo ===")

    queue = asyncio.Queue(maxsize=5)

    # Schedule one producer and two consumers concurrently
    prod_task = asyncio.create_task(async_producer(queue, 10))
    cons_tasks = [asyncio.create_task(async_consumer(queue, f"C{i}")) for i in (1,2)]

    # Wait for producer to finish
    await prod_task

    # Wait for consumers to finish
    await asyncio.gather(*cons_tasks)

    print("Async demo completed!")

# Run the async demo
asyncio.run(async_producer_consumer_demo())

In [ ]:
import asyncio
import aiohttp  # Note: you may need to install aiohttp
import time

# Async HTTP client example
async def fetch_url(session, url):
    """Fetch a single URL asynchronously"""
    try:
        async with session.get(url) as response:
            content = await response.text()
            return url, response.status, len(content)
    except Exception as e:
        return url, "ERROR", str(e)

async def async_http_demo():
    """Demo of concurrent HTTP requests using async"""
    print("=== Async HTTP Requests Demo ===")

    # Test URLs (using httpbin for reliable testing)
    urls = [
        "https://httpbin.org/delay/1",
        "https://httpbin.org/delay/2",
        "https://httpbin.org/delay/1",
        "https://httpbin.org/status/200",
        "https://httpbin.org/json"
    ]

    start_time = time.time()

    async with aiohttp.ClientSession() as session:
        # Start all requests concurrently
        tasks = [fetch_url(session, url) for url in urls]
        results = await asyncio.gather(*tasks)

    end_time = time.time()

    for url, status, content_info in results:
        print(f"{url} -> Status: {status}, Content: {content_info}")

    print(f"Total time: {end_time - start_time:.2f}s")
    print("Note: This would take ~6s sequentially, but much less concurrently!")

# Uncomment to run (requires aiohttp installation)
# asyncio.run(async_http_demo())

print("HTTP demo code ready (install aiohttp to run)")

In [ ]:
# Advanced async patterns: semaphores, locks, and error handling
import asyncio
import random

async def limited_resource_demo():
    """Demo using semaphores to limit concurrent resource access"""
    print("=== Async Semaphore Demo ===")

    # Limit to 2 concurrent "connections"
    semaphore = asyncio.Semaphore(2)

    async def access_limited_resource(resource_id):
        async with semaphore:  # Acquire semaphore
            print(f"Resource {resource_id} acquired semaphore")
            await asyncio.sleep(random.uniform(1, 3))  # Simulate work
            print(f"Resource {resource_id} releasing semaphore")
            return f"Resource {resource_id} completed"

    # Start 5 tasks, but only 2 can run concurrently
    tasks = [access_limited_resource(i) for i in range(5)]
    results = await asyncio.gather(*tasks)

    for result in results:
        print(f"✅ {result}")

async def async_lock_demo():
    """Demo using async locks for shared state"""
    print("\n=== Async Lock Demo ===")

    shared_counter = 0
    lock = asyncio.Lock()

    async def increment_counter(worker_id, increments):
        nonlocal shared_counter

        for i in range(increments):
            async with lock:  # Critical section
                current = shared_counter
                await asyncio.sleep(0.01)  # Simulate work that could cause race condition
                shared_counter = current + 1
                print(f"Worker {worker_id}: counter = {shared_counter}")

    # Start multiple workers
    tasks = [increment_counter(i, 3) for i in range(3)]
    await asyncio.gather(*tasks)

    print(f"Final counter value: {shared_counter}")

async def error_handling_demo():
    """Demo proper error handling in async code"""
    print("\n=== Async Error Handling Demo ===")

    async def task_that_might_fail(task_id):
        await asyncio.sleep(random.uniform(0.1, 0.5))
        if random.random() < 0.4:  # 40% chance of failure
            raise ValueError(f"Task {task_id} failed!")
        return f"Task {task_id} succeeded"

    tasks = [task_that_might_fail(i) for i in range(5)]

    # Method 1: Catch all exceptions with return_exceptions=True
    results = await asyncio.gather(*tasks, return_exceptions=True)

    for i, result in enumerate(results):
        if isinstance(result, Exception):
            print(f"❌ Task {i}: {result}")
        else:
            print(f"✅ Task {i}: {result}")

# Run all demos
async def main_async_demos():
    await limited_resource_demo()
    await async_lock_demo()
    await error_handling_demo()

asyncio.run(main_async_demos())

## Performance Comparison: Threads vs Async vs Processes

Let's compare the different concurrency approaches for different types of tasks:

In [ ]:
import time
import asyncio
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor

def io_bound_task(duration):
    """Simulate I/O-bound work"""
    time.sleep(duration)
    return f"I/O task completed in {duration}s"

async def async_io_task(duration):
    """Async version of I/O-bound work"""
    await asyncio.sleep(duration)
    return f"Async I/O task completed in {duration}s"

def cpu_bound_task(n):
    """Simulate CPU-bound work"""
    total = sum(i * i for i in range(n))
    return f"CPU task calculated sum: {total}"

def compare_approaches():
    """Compare different concurrency approaches"""
    print("=== Performance Comparison ===")

    # Test parameters
    io_durations = [0.5, 0.8, 0.3, 0.6, 0.4]  # I/O tasks
    cpu_values = [100000, 150000, 120000, 180000]  # CPU tasks

    # 1. Sequential (baseline)
    print("\n1. Sequential I/O:")
    start = time.time()
    for duration in io_durations:
        io_bound_task(duration)
    sequential_time = time.time() - start
    print(f"Time: {sequential_time:.2f}s")

    # 2. ThreadPool for I/O
    print("\n2. ThreadPool I/O:")
    start = time.time()
    with ThreadPoolExecutor(max_workers=5) as executor:
        results = list(executor.map(io_bound_task, io_durations))
    thread_time = time.time() - start
    print(f"Time: {thread_time:.2f}s")
    print(f"Speedup: {sequential_time / thread_time:.2f}x")

    # 3. Async for I/O
    print("\n3. Async I/O:")
    async def async_test():
        start = time.time()
        tasks = [async_io_task(duration) for duration in io_durations]
        await asyncio.gather(*tasks)
        return time.time() - start

    async_time = asyncio.run(async_test())
    print(f"Time: {async_time:.2f}s")
    print(f"Speedup: {sequential_time / async_time:.2f}x")

    # 4. CPU-bound comparison
    print("\n4. CPU-bound tasks:")

    # Sequential CPU
    start = time.time()
    for val in cpu_values:
        cpu_bound_task(val)
    cpu_sequential = time.time() - start
    print(f"Sequential CPU: {cpu_sequential:.2f}s")

    # ThreadPool CPU (limited by GIL)
    start = time.time()
    with ThreadPoolExecutor(max_workers=4) as executor:
        list(executor.map(cpu_bound_task, cpu_values))
    cpu_thread = time.time() - start
    print(f"ThreadPool CPU: {cpu_thread:.2f}s (GIL limited)")

    # ProcessPool CPU (can use multiple cores)
    start = time.time()
    with ProcessPoolExecutor(max_workers=4) as executor:
        list(executor.map(cpu_bound_task, cpu_values))
    cpu_process = time.time() - start
    print(f"ProcessPool CPU: {cpu_process:.2f}s")
    print(f"Process speedup: {cpu_sequential / cpu_process:.2f}x")

compare_approaches()

## Summary

Concurrency and parallelism patterns in Python provide powerful tools for improving performance:

### Key Patterns Covered:

1. **Producer-Consumer Pattern**: Decouples data production and consumption using thread-safe queues
2. **Thread Pool Pattern**: Reuses threads for multiple tasks, ideal for I/O-bound operations
3. **Async/Await Pattern**: Single-threaded cooperative multitasking for high-concurrency I/O

### When to Use What:

- **Threading (`ThreadPoolExecutor`)**:
  - ✅ I/O-bound tasks (file operations, network requests)
  - ❌ CPU-bound tasks (limited by GIL)
  - Simple to use, good for moderate concurrency

- **Async/Await (`asyncio`)**:
  - ✅ High-concurrency I/O (thousands of connections)
  - ✅ Single-threaded, no race conditions
  - ❌ Requires async ecosystem (aiohttp, asyncpg, etc.)
  - Steeper learning curve but very efficient

- **Multiprocessing (`ProcessPoolExecutor`)**:
  - ✅ CPU-bound tasks (mathematical computations)
  - ✅ True parallelism (bypasses GIL)
  - ❌ Higher memory overhead
  - ❌ Data serialization costs between processes

### Best Practices:

- Use context managers (`with` statements) for proper resource cleanup
- Handle exceptions properly in concurrent code
- Use semaphores and locks to control resource access
- Consider the nature of your tasks (I/O vs CPU-bound) when choosing approach
- Profile and measure - concurrency doesn't always improve performance